In [20]:
import numpy as np
import pandas as pd

from classes_for_df_preparation import common_data, gender_parce

In [46]:
!python test_for_cls.py -v

test_par (__main__.full_test.test_par) ... ERROR

ERROR: test_par (__main__.full_test.test_par)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\nikgo\Documents\NSU\Theory_of_stat_decisions\Theory_of_stat_decisions\lab03\test_for_cls.py", line 25, in test_par
    self.assertNotEqual(gender_parce(1, df), df_t)
  File "c:\Users\nikgo\AppData\Local\Programs\Python\Python311\Lib\unittest\case.py", line 879, in assertNotEqual
    if not first != second:
           ^^^^^^^^^^^^^^^
  File "c:\Users\nikgo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\generic.py", line 1466, in __nonzero__
    raise ValueError(
ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (errors=1)


In [44]:
df = pd.read_csv('./data/survey.csv')
df = df.drop(columns=['state', 'Country', 'Timestamp', 'comments', 'no_employees'])
df['Gender'] = gender_parce(df.shape[0], df['Gender'])
print(df)

c:\Users\nikgo\Documents\NSU\Theory_of_stat_decisions\Theory_of_stat_decisions\lab03\classes_for_df_preparation.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_copy[row] = dict_fot_col[sym]
c:\Users\nikgo\Documents\NSU\Theory_of_stat_decisions\Theory_of_stat_decisions\lab03\classes_for_df_preparation.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_copy[row] = dict_fot_col[sym]
c:\Users\nikgo\Documents\NSU\Theory_of_stat_decisions\Theory_of_stat_decisions\lab03\classes_for_df_preparation.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

KeyError: 'T'

(20, 7)
